# Projet Data viz

Dans le cadre de ce projet, nous avons notre propre base de données en nous appuyant sur l'API de Yahoo Finance grâce au package Python `yfinance`. L'objectif principal était de collecter des données financières détaillées sur les entreprises qui composent l'indice S&P 500, ainsi que des informations historiques sur l'indice lui-même.

## Collecte des Tickers et Construction de la Base

Pour identifier les entreprises du S&P 500, j'ai avons utilisé une base de données en accès libre disponible sur le site DataHub.io. Cette ressource fournit une liste complète des tickers des entreprises cotées au sein de cet indice, accompagnée de leurs secteurs et sous-secteurs d'activité (GICS). Ces tickers sont essentiels pour accéder aux données financières via l'API.

Lien vers la base de données utilisée :
https://datahub.io/core/s-and-p-500-companies/

Le fichier source contient les informations suivantes (entre-autres) :

- **Symbol :** Le ticker boursier (ou symbole) unique de chaque entreprise.
- **Security :** Le nom complet de l'entreprise.
- **GICS Sector :** Le secteur d'activité auquel l'entreprise appartient.
- **GICS Sub-Industry :** Le sous-secteur d'activité plus spécifique.

Avant d'utiliser ces tickers dans le code, j'ai remplacé les points (.) présents dans certains tickers par des tirets (-) afin de respecter le format attendu par Yahoo Finance.

## Implémentation en Python pour Importer les Données

### Description du Code : Téléchargement et Préparation des Données du S&P 500

Ce script a pour objectif de télécharger les données historiques des entreprises composant l'indice S&P 500, de les enrichir avec des métadonnées, puis de fusionner ces données avec l'historique de l'indice S&P 500. Enfin, les données consolidées sont exportées dans un fichier CSV.

---

#### 1. Importation des Librairies et des Données Initiales

- **`pandas`** : Utilisé pour manipuler les données tabulaires et effectuer des opérations comme la lecture et l’écriture de fichiers CSV.
- **`yfinance`** : Utilisé pour interagir avec l’API de Yahoo Finance et récupérer des données financières historiques.

Le fichier CSV contenant les informations des entreprises du S&P 500 est téléchargé depuis [DataHub.io](https://datahub.io/core/s-and-p-500-companies/). Ce fichier contient :
- **Symbol** : Ticker boursier unique de chaque entreprise.
- **Security** : Nom de l'entreprise.
- **GICS Sector** : Secteur d'activité.
- **GICS Sub-Industry** : Sous-secteur d'activité.

Une étape de prétraitement ajuste les tickers pour remplacer les points (`.`) par des tirets (`-`), ce qui correspond au format attendu par Yahoo Finance.

---

#### 2. Fonction pour Interroger l'API de Yahoo Finance

Une fonction **`api_request(a, b, c, d)`** est définie pour :
- **`a`** : Ticker de l'action ou de l'indice.
- **`b`**, **`c`** : Dates de début et de fin pour la récupération des données.
- **`d`** : Fréquence des données (ex. : journalière, horaire, etc.).

Elle retourne un tableau contenant les données de prix historiques pour le ticker spécifié.

---

#### 3. Téléchargement des Données Historiques de l'Indice S&P 500

Les données de l’indice S&P 500, identifiées par le ticker `^GSPC`, sont téléchargées pour la période allant du 1er novembre 2019 au 1er novembre 2024 avec une fréquence quotidienne. Une fois récupérées, elles sont transformées pour ne conserver que deux colonnes :
- **Date** : Date des données.
- **SP500 Close** : Prix de clôture de l’indice.

Ces données serviront de référence (benchmark) pour les entreprises du S&P 500.

---

#### 4. Téléchargement des Données des Entreprises du S&P 500

Une boucle parcourt chaque ligne du fichier des constituants du S&P 500 :
1. **Extraction des informations par entreprise** : 
   - **`Symbol`** : Ticker de l'entreprise.
   - **`Security`** : Nom de l'entreprise.
   - **`GICS Sector`** et **`GICS Sub-Industry`** : Métadonnées sur le secteur et sous-secteur d'activité.
   
2. **Récupération des données historiques via l’API** :  
   Les données historiques de l’action sont récupérées sur la même période que pour l’indice S&P 500 (1er novembre 2019 au 1er novembre 2024).

3. **Ajout des métadonnées** :  
   Les colonnes suivantes sont ajoutées aux données de l’action pour enrichir les informations :
   - **Security** : Nom complet de l'entreprise.
   - **GICS Sector** : Secteur d’activité.
   - **GICS Sub-Industry** : Sous-secteur d’activité.
   - **Symbol** : Ticker de l’entreprise.

4. **Fusion des données** :  
   Les données de chaque entreprise sont ajoutées au DataFrame principal **`data`** à l’aide de la fonction **`pd.concat`**.

Un compteur affiche l’état d’avancement de la boucle pour suivre la progression.

---

#### 5. Fusion avec les Données de l'Indice S&P 500

Une fusion est effectuée entre les données des entreprises du S&P 500 et celles de l'indice S&P 500, en utilisant la colonne **Date** comme clé commune. Cette étape permet d’ajouter la valeur de clôture de l’indice à chaque ligne correspondant à une entreprise.

---

#### 6. Exportation des Données Consolidées

- Les données fusionnées et enrichies sont exportées dans un fichier CSV nommé **`imported_data.csv`**.
- Le fichier est structuré avec des colonnes contenant les données historiques des prix, les métadonnées des entreprises, et la valeur de clôture de l’indice S&P 500.

---

### Résumé des Étapes
1. Téléchargement des informations des entreprises du S&P 500 depuis une base de données en ligne.
2. Récupération des données historiques des entreprises et de l'indice S&P 500 via l'API Yahoo Finance.
3. Ajout de métadonnées (secteur, sous-secteur, etc.) pour chaque entreprise.
4. Fusion des données avec l'indice S&P 500 pour inclure un benchmark.
5. Exportation des données consolidées dans un fichier CSV.

Ce script génère une base de données prête pour une analyse approfondie des performances des actions du S&P 500 sur une période donnée.


In [ ]:
import pandas as pd
import yfinance as yf

constituents = pd.read_csv("https://datahub.io/core/s-and-p-500-companies/_r/-/data/constituents.csv")
constituents["Symbol"] = constituents["Symbol"].str.replace(r"\.", "-", regex=True)
cn = len(constituents)

def api_request(a, b, c, d):
    x = yf.Ticker(a)
    y = x.history(start=b, end=c, interval=d)
    return y

data = pd.DataFrame()

print("Téléchargement des données du S&P 500...")
sp500 = api_request("^GSPC", "2019-11-01", "2024-11-01", "1d")
sp500 = sp500.reset_index()
sp500 = sp500[["Date", "Close"]].rename(columns={"Close": "SP500 Close"})

for i, row in constituents.iterrows():
    symbol = row["Symbol"]
    security = row["Security"]
    sector = row["GICS Sector"]
    industry = row["GICS Sub-Industry"]

    print(f"[{i + 1}/{cn}] {symbol}...", end="")
    history = api_request(symbol, "2019-11-01", "2024-11-01", "1d")
    print(" X")
    history = history.reset_index()

    history.insert(0, "Security", security)
    history["GICS Sector"] = sector
    history["GICS Sub-Industry"] = industry
    history["Symbol"] = symbol

    data = pd.concat([data, history])


print("Fusion des données avec le S&P 500...")
data["Date"] = pd.to_datetime(data["Date"])
sp500["Date"] = pd.to_datetime(sp500["Date"])
data = pd.merge(data, sp500, on="Date", how="left")

print()
print(50 * "*")
print("Données en cours d'exportation...")
data.to_csv("imported_data.csv", sep=";", index=False)
print("Données exportées avec succès !")
print(50 * "*")

### Nettoyage et Enrichissement des Données : Explication du Code

Le code présenté effectue un nettoyage approfondi des données et les enrichit avec une variété d'indicateurs financiers avancés pour évaluer les performances et la volatilité des actions. Voici une explication détaillée des différentes étapes et calculs réalisés :

---

#### Importation des Données
- Les données sont importées à partir du fichier CSV **`imported_data.csv`**, contenant les informations sur les actions et l'indice S&P 500.

---

#### Calcul des Indicateurs de Base

1. **Performance Intraday (%)** :  
   $$
   \text{Performance Intraday (\%)} = \frac{\text{Close} - \text{Open}}{\text{Open}} \times 100
   $$

2. **Moyenne Mobile sur 30 Jours (USD)** :  
   $$
   \text{30-Day Moving Average (USD)} = \text{Rolling Mean (Close, 30 jours)}
   $$

---

#### Calcul des Rendements et des Rendements Excédentaires

1. **Rendement de l'Actif** :  
   $$
   \text{Asset Return} = \text{pct\_change}(\text{Close})
   $$

2. **Rendement de l'Indice S&P 500** :  
   $$
   \text{Benchmark Return} = \text{pct\_change}(\text{SP500 Close})
   $$

3. **Rendement Excédentaire** :  
   $$
   \text{Excess Return} = \text{Asset Return} - \text{Benchmark Return}
   $$

---

#### Calcul du Ratio d'Information

Le **Ratio d'Information** est calculé comme suit pour chaque groupe d'actions :
$$
\text{Information Ratio} = \frac{\text{Moyenne des Rendements Excédentaires}}{\text{Écart-Type des Rendements Excédentaires}}
$$

---

#### Conversion des Rendements en Pourcentages

- Les colonnes des rendements ($\text{Asset Return}$, $\text{Benchmark Return}$, $\text{Excess Return}$) sont multipliées par 100 pour obtenir des pourcentages.

---

#### Calcul du Beta Glissant (30 Jours)

Le **Rolling Beta** (Beta sur une période glissante de 30 jours) est calculé pour chaque groupe d'actions comme suit :
$$
\text{Rolling Beta (30 days)} = \frac{\text{Rolling Cov}(\text{Asset Return}, \text{Benchmark Return})}{\text{Rolling Var}(\text{Benchmark Return})}
$$

---

#### Ajout de Mesures de Risque

1. **Max Drawdown sur 30 Jours** :  
   Le **Drawdown** mesure la perte maximale par rapport au point le plus haut atteint dans une fenêtre glissante de 30 jours :
   $$
   \text{Max Drawdown} = \min\left(\frac{\text{Valeurs Cumulées}}{\max(\text{Valeurs Cumulées})} - 1\right)
   $$

2. **VaR (5%) sur 30 Jours** :  
   La **Value at Risk (VaR)** correspond au rendement minimum à un niveau de confiance de 5% dans une fenêtre de 30 jours :
   $$
   \text{VaR (5\%)} = \text{Percentile à 5\% des Rendements dans la Fenêtre}
   $$

---

#### Calcul des Performances Cumulées YTD (Year-To-Date)

Pour chaque action, la performance cumulée depuis le début de l'année en cours est calculée :
$$
\text{YTD Performance (\%)} = \frac{\text{Close} - \text{First Close of the Year}}{\text{First Close of the Year}} \times 100
$$

---

#### Renommage et Nettoyage des Données

Les colonnes sont renommées pour une meilleure lisibilité, et certaines colonnes inutiles sont supprimées. Par exemple :  
- Renommage des colonnes liées aux prix :  
  - `Open` → **`Price Open (USD)`**
  - `Close` → **`Price Close (USD)`**
  - `SP500 Close` → **`S&P500 Close (USD)`**

---

#### Calcul d'Indicateurs Complémentaires

1. **Amplitude des Prix (USD)** :  
   $$
   \text{Price Range (USD)} = \text{Price High (USD)} - \text{Price Low (USD)}
   $$

2. **Variation des Prix (USD et %)** :  
   - En valeur absolue :  
     $$
     \text{Price Change (USD)} = \text{Price Close (USD)} - \text{Price Open (USD)}
     $$
   - En pourcentage :  
     $$
     \text{Price Change (\%)} = \frac{\text{Price Change (USD)}}{\text{Price Open (USD)}} \times 100
     $$

3. **Volatilité (%)** :  
   $$
   \text{Volatility (\%)} = \frac{\text{Price High (USD)} - \text{Price Low (USD)}}{\text{Price Low (USD)}} \times 100
   $$

4. **Rendement du Dividende (%)** :  
   $$
   \text{Dividend Yield (\%)} = \frac{\text{Dividends}}{\text{Price Close (USD)}} \times 100
   $$

5. **Prix Moyen (USD)** :  
   $$
   \text{Average Price (USD)} = \frac{\text{Price Open (USD)} + \text{Price Close (USD)}}{2}
   $$

6. **Ratio Volume/Prix** :  
   $$
   \text{Volume to Price Ratio} = \frac{\text{Volume}}{\text{Price Close (USD)}}
   $$

---

#### Exportation des Données

Les données nettoyées et enrichies sont exportées dans un fichier CSV nommé **`data.csv`**, indexé par le ticker et la date, pour faciliter les analyses futures.



In [ ]:
import pandas as pd
import numpy as np

data = pd.read_csv("imported_data.csv", sep=";")

data["Performance Intraday (%)"] = ((data["Close"] - data["Open"]) / data["Open"]) * 100
data["30-Day Moving Average (USD)"] = data["Close"].rolling(window=30).mean()


data["Asset Return"] = data.groupby("Symbol")["Close"].pct_change()  # Rendements de l'actif
data["Benchmark Return"] = data["SP500 Close"].pct_change()         # Rendements du S&P 500


data["Excess Return"] = data["Asset Return"] - data["Benchmark Return"]


def calculate_information_ratio(group):
    mean_excess_return = group["Excess Return"].mean()  # Moyenne des rendements excédentaires
    tracking_error = group["Excess Return"].std()       # Écart-type des rendements excédentaires
    if tracking_error == 0:
        return np.nan  # Éviter la division par zéro
    return mean_excess_return / tracking_error


information_ratios = data.groupby("Symbol").apply(calculate_information_ratio)
information_ratios.name = "Information Ratio"


data = data.merge(information_ratios, on="Symbol", how="left")

data["Asset Return (%)"] = data["Asset Return"] * 100
data["Benchmark Return (%)"] = data["Benchmark Return"] * 100
data["Excess Return (%)"] = data["Excess Return"] * 100
columns_to_drop = ["Asset Return", "Benchmark Return", "Excess Return", "Information Ratio_x", "Information Ratio_y"]
data = data.drop(columns=columns_to_drop, errors="ignore")

data["Asset Return"] = data["Asset Return (%)"] / 100  # Convert to fraction
data["Benchmark Return"] = data["Benchmark Return (%)"] / 100  # Convert to fraction

def calculate_rolling_beta(group):
    rolling_cov = group["Asset Return"].rolling(window=30).cov(group["Benchmark Return"])
    rolling_var = group["Benchmark Return"].rolling(window=30).var()
    return rolling_cov / rolling_var

data["Rolling Beta (30 days)"] = (
    data.groupby("Symbol", group_keys=False)
    .apply(calculate_rolling_beta)
)


def add_max_drawdown_30_days(data):

    column_returns = "Asset Return (%)"
    max_drawdown_column_name = "Max Drawdown (30 days)"
    window_size = 30

    if column_returns not in data.columns:
        raise ValueError(f"La colonne '{column_returns}' n'existe pas dans le DataFrame.")


    def calculate_max_drawdown_window(returns):
        if len(returns) < window_size: 
            return np.nan
        
        cumulative_returns = np.cumprod(1 + returns / 100) 
        
        drawdown = (cumulative_returns / np.maximum.accumulate(cumulative_returns)) - 1
        return drawdown.min()  # Drawdown le plus sévère


    try:
        data[max_drawdown_column_name] = (
            data.groupby("Symbol")[column_returns]
            .rolling(window=window_size)
            .apply(calculate_max_drawdown_window, raw=True)
            .reset_index(level=0, drop=True)
        )
    except Exception as e:
        print(f"Erreur lors du calcul du Max Drawdown : {e}")

    return data


def add_var_30_days(data):

    column_returns = "Asset Return (%)"
    var_column_name = "VaR (5%) (30 days)"
    window_size = 30
    confidence_level = 5

    if column_returns not in data.columns:
        raise ValueError(f"La colonne '{column_returns}' n'existe pas dans le DataFrame.")


    def calculate_var_window(returns):
        if len(returns) < window_size:
            return np.nan
        return np.percentile(returns, confidence_level)


    data[var_column_name] = (
        data.groupby("Symbol")[column_returns]
        .rolling(window=window_size)
        .apply(calculate_var_window, raw=True)
        .reset_index(level=0, drop=True)
    )

    return data

data = add_max_drawdown_30_days(data)
data = add_var_30_days(data)

def add_daily_ytd(data):

    data["Date"] = pd.to_datetime(data["Date"], utc=True)
    
    data = data.sort_values(["Symbol", "Date"])
    
    def calculate_daily_ytd(group):

        group["Year"] = group["Date"].dt.year
        first_close_per_year = group.groupby("Year")["Close"].transform("first")
        
        group["YTD Performance (%)"] = ((group["Close"] - first_close_per_year) / first_close_per_year) * 100
        
        if group["Year"].min() == data["Date"].dt.year.min():
            group.loc[group["Year"] == data["Date"].dt.year.min(), "YTD Performance (%)"] = np.nan
        
        return group

    data = data.groupby("Symbol", group_keys=False).apply(calculate_daily_ytd)
    data = data.drop(columns=["Year"])  # Clean up temporary column

    return data

    
data = add_daily_ytd(data)

def get_data():
    return data

data = data.rename(columns={
    "Symbol": "Ticker",
    "Open": "Price Open (USD)",
    "High": "Price High (USD)",
    "Low": "Price Low (USD)",
    "Close": "Price Close (USD)",
    "SP500 Close": "S&P500 Close (USD)"
    })

data = data.drop(columns=["Stock Splits"])
data = data.drop(columns=["Asset Return"])
data = data.drop(columns=["Benchmark Return"])

data["Price Range (USD)"] = data["Price High (USD)"] - data["Price Low (USD)"]
data["Price Change (USD)"] = data["Price Close (USD)"] - data["Price Open (USD)"]
data["Price Change (%)"] = (data["Price Close (USD)"] - data["Price Open (USD)"]) / data["Price Open (USD)"] * 100
data["Volatility (%)"] = (data["Price High (USD)"] - data["Price Low (USD)"]) / data["Price Low (USD)"] * 100
data["Dividend Yield (%)"] = data["Dividends"] / data["Price Close (USD)"] * 100
data["Average Price (USD)"] = (data["Price Open (USD)"] + data["Price Close (USD)"]) / 2
data["Volume to Price Ratio"] = data["Volume"] / data["Price Close (USD)"]

data["Date"] = pd.to_datetime(data["Date"], utc=True)
data["Date"] = data["Date"].dt.strftime('%Y-%m-%d')

data = data.sort_values(by=["Ticker", "Date"])
data = data.set_index(["Ticker", "Date"])

print()
print(50 * "*")
print("Données en cours d'exportation...")
data.to_csv("data.csv", sep=";", index=True)
print("Données exportées avec succès !")
print(50 * "*")

Au total, nous avons 624 247 uniques observations pour 503 entreprises côtées en bourse.